In [35]:
from numpy import exp, dot, conj
from numpy.linalg import norm
import numpy as np 
import matplotlib.pyplot as plt
from numpy import trapz as Int
from collections import namedtuple

# Inner terms

First we are going to check the inner terms. There are $4$ situations:
 - $\phi^+_n$ and $\psi^+_m$: 
$$
\frac{1}{2}\int_E \phi_n\overline{\nabla \psi_m\cdot\mathbf{n}}-\nabla \phi_n\cdot\mathbf{n}\overline{\psi_m}\,\mathrm{d}S_x+a\dots+b\dots
$$
 - $\phi^+_n$ and $\psi^-_m$: 
$$
-\frac{1}{2}\int_E \phi_n\overline{\nabla \psi_m\cdot\mathbf{n}}-\nabla \phi_n\cdot\mathbf{n}\overline{\psi_m}\,\mathrm{d}S_x+a\dots+b\dots
$$
 - $\phi^-_n$ and $\psi^+_m$: 
$$
\frac{1}{2}\int_E \phi_n\overline{\nabla \psi_m\cdot\mathbf{n}}-\nabla \phi_n\cdot\mathbf{n}\overline{\psi_m}\,\mathrm{d}S_x+a\dots+b\dots
$$
 - $\phi^-_n$ and $\psi^-_m$: 
$$
-\frac{1}{2}\int_E \phi_n\overline{\nabla \psi_m\cdot\mathbf{n}}-\nabla \phi_n\cdot\mathbf{n}\overline{\psi_m}\,\mathrm{d}S_x++a\dots+b\dots
$$


For checking the central fluxes only one expression needs to be checked: 

$$
\frac{1}{2}\int_E \varphi_n\overline{\nabla \psi_m\cdot\mathbf{n}}-\nabla \varphi_n\cdot\mathbf{n}\overline{\psi_m}\,\mathrm{d}S_x
$$

Assuming $\varphi_n(\mathbf{x})=e^{ik\mathbf{d}_n\cdot\mathbf{x}}$ and $\psi_m(\mathbf{x})=e^{ik\mathbf{d}_m\cdot\mathbf{x}}$, the term above becomes:
$$
-ikl\left(\mathbf{d}_{m}+\mathbf{d}_{n}\right)\cdot\mathbf{n}\frac{e^{ik\left(\mathbf{d}_{n}-\mathbf{d}_{m}\right)\cdot\mathbf{P}}}{2}\qquad\text{if}\quad\mathbf{d}_n\cdot\boldsymbol{\tau}=\mathbf{d}_m\cdot\boldsymbol{\tau}
$$
$$
-\frac{\left(\mathbf{d}_{m}+\mathbf{d}_{n}\right)\cdot\mathbf{n}}{\left(\mathbf{d}_{n}-\mathbf{d}_{m}\right)\cdot\boldsymbol{\tau}}\frac{e^{ik\left(\mathbf{d}_{n}-\mathbf{d}_{m}\right)\cdot\mathbf{Q}}-e^{ik\left(\mathbf{d}_{n}-\mathbf{d}_{m}\right)\cdot\mathbf{P}}}{2}\qquad\text{otherwise}
$$

## Exact term

In [36]:
def Inner_term_PP(phi, psi, edge, k, a, b):

    d_m = psi.d
    d_n = phi.d
    
    P = edge.P
    Q = edge.Q
    N = edge.N
    T = edge.T

    l = norm(Q-P)

    tol = 1E-6

    I = dot( d_m, N) + dot( d_n, N) + 2*b*dot( d_m, N)*dot( d_n, N) + 2*a


    if np.isclose( dot(d_m,T), dot(d_n,T), tol) :
        return -1/2*1j*k*l * I * exp(1j*k*dot(d_n - d_m, P))
    else:
        return -1/2*I/dot(d_n - d_m, T)*( exp(1j*k*dot(d_n - d_m, Q)) - exp(1j*k*dot(d_n - d_m, P)))

## Numerical term (a,b=0)

In [54]:
def num_inner( k, P, Q, N, d_n, d_m, a=0, b=0, Nt = 100):
    Px, Py = P[0], P[1]
    Qx, Qy = Q[0], Q[1]
    l = norm(Q-P)
    t = np.linspace(0,1,Nt)
    x = P + np.outer(t,Q-P)
    phi_n = exp(1j*k*dot(x,d_n))
    psi_m = exp(1j*k*dot(x,d_m))
    grad_phi_n_N = 1j*k*dot(N,d_n)*exp(1j*k*dot(x,d_n))
    grad_psi_m_N = 1j*k*dot(N,d_m)*exp(1j*k*dot(x,d_m))

    I = l/2*Int(phi_n*conj(grad_psi_m_N) - grad_phi_n_N*conj(psi_m), t)
    I+= l*b*Int(1/(1j*k)*grad_phi_n_N * conj(grad_psi_m_N), t)
    I-= l*a*Int(1j*k*phi_n*psi_m)


    return I



In [55]:
# Test
P = np.array([0,0])
Q = np.array([1,1])

T = (Q - P)/norm(Q-P)
N = np.array([-T[1], T[0]])

Edge = namedtuple('Edge',['P','Q','N','T'])
E = Edge(P,Q,N,T)


k = 8.
d_n = np.array([1,1])/norm([1,1])
d_m = np.array([1,-1])/norm([1,-1])

TestFunction = namedtuple('TestFunction',['d','k'])
phi_n = TestFunction(d=d_n,k=k)
psi_m = TestFunction(d=d_m,k=k)

a = 0.5
b = 0.5


I_exact = Inner_term_PP(phi_n, psi_m, E, k, 0., 0.)
I_num = num_inner( k, P, Q, N, d_n, d_m, a = a, b = b,  Nt=int(1E4))
relative_error = abs(I_exact - I_num)/abs(I_exact)
print(f'I_exact: {I_exact:.16f}\nI_num:   {I_num:.16f}\nRelative error: {relative_error :.2e}')

I_exact: -0.3436025292439285-0.4749103401075868j
I_num:   -0.3436024925856599-0.4749102894403498j
Relative error: 1.07e-07
